In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [2]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.sample(10)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
371,24/11/2020 18:39:13,41.0,Female,Munich,Data Engineer,12,12,Senior,NaN,"Python, SQL, Azure",68000.0,4500,30,Full-time employee,Unlimited contract,German,101-1000,Consulting / Agency
1095,05/12/2020 17:29:17,28.0,Female,Berlin,Backend Developer,3,2,Middle,Python,"AWS, Docker",65000.0,0,30,Full-time employee,Unlimited contract,English,1000+,Product
147,24/11/2020 12:10:05,31.0,Female,Munich,QA Engineer,7,3,Senior,Java,"Python, Ruby",68000.0,NaN,30,Full-time employee,Unlimited contract,English,1000+,Product
875,27/11/2020 21:00:34,33.0,Male,Munich,Frontend Developer,14,8,Senior,"Angular, Typescript","PHP, Javascript / Typescript",80000.0,NaN,28,Full-time employee,Unlimited contract,German,11-50,Product
21,24/11/2020 11:21:18,34.0,Male,Berlin,Software Architect,15,9,Lead,NodeJS,"Javascript / Typescript, SQL, Go, AWS, Kuberne...",100000.0,NaN,24,Full-time employee,Unlimited contract,English,1000+,Product
408,24/11/2020 20:02:34,30.0,Male,Munich,Mobile Developer,8,2,Lead,Swift,Swift,75000.0,86000,24,Full-time employee,Unlimited contract,English,11-50,Startup
740,26/11/2020 09:02:41,30.0,Male,Berlin,Backend Developer,8,2,Middle,Python,"SQL, AWS, Kubernetes, Docker",74000.0,11000,27,Full-time employee,Unlimited contract,English,1000+,Product
443,24/11/2020 21:38:24,24.0,Male,Berlin,Backend Developer,3,0,Junior,C#,"Javascript / Typescript, .NET, Kubernetes, Docker",48000.0,NaN,28,Full-time employee,Temporary contract,English,1000+,Game Company
563,25/11/2020 12:23:50,30.0,Female,Berlin,Data Scientist,6,5,Senior,python,"Python, SQL, Google Cloud, Kubernetes, Docker",76000.0,10000,27,Full-time employee,Unlimited contract,English,1000+,Product
279,24/11/2020 15:05:21,39.0,Male,Berlin,Product Manager,10,5,Lead,NaN,NaN,120000.0,200000,25,Full-time employee,Unlimited contract,English,1000+,Product


In [4]:
#df.info()
df["Other technologies/programming languages you use often"].value_counts()

Javascript / Typescript                                                                   44
Python                                                                                    37
SQL                                                                                       31
AWS, Docker                                                                               16
Kotlin                                                                                    15
                                                                                          ..
Python, Ruby, Java / Scala, Go, Rust, AWS, Kubernetes, Docker, Terraform, helm, packer     1
Kotlin, SQL, AWS, Kubernetes, Docker                                                       1
Python, R, Go                                                                              1
C/C++, PHP, SQL                                                                            1
Python, AWS, Kubernetes, Docker, terraform, ansible                   

In [5]:
df.head()

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
0,24/11/2020 11:14:15,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.0,5000,30,Full-time employee,Unlimited contract,English,51-100,Product
1,24/11/2020 11:14:16,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,80000.0,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product
2,24/11/2020 11:14:21,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",120000.0,120000,30,Self-employed (freelancer),Temporary contract,English,101-1000,Product
3,24/11/2020 11:15:24,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,54000.0,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup
4,24/11/2020 11:15:46,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",62000.0,NaN,29,Full-time employee,Unlimited contract,English,101-1000,Product


# Answers and Explainations
(Expand/modify as needed)

### Results

### Feature Selection Activities

### Hyperparameter Changes

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 18 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Timestamp                                               1253 non-null   object 
 1   Age                                                     1226 non-null   float64
 2   Gender                                                  1243 non-null   object 
 3   City                                                    1253 non-null   object 
 4   Position                                                1247 non-null   object 
 5   Total years of experience                               1237 non-null   object 
 6   Years of experience in Germany                          1221 non-null   object 
 7   Seniority level                                         1241 non-null   object 
 8   Your main technology / programming lan

In [7]:
# Check for nulls
df.isnull().sum().sort_values()

Timestamp                                                   0
City                                                        0
Yearly brutto salary (without bonus and stocks) in EUR      0
Position                                                    6
Gender                                                     10
Seniority level                                            12
Main language at work                                      16
Total years of experience                                  16
Employment status                                          17
Company size                                               18
Company type                                               25
Age                                                        27
Сontract duration                                          29
Years of experience in Germany                             32
Number of vacation days                                    68
Your main technology / programming language               127
Other te

In [8]:
# Combinding columns and changing dtypes
df["Bonus"] = pd.to_numeric(df["Yearly bonus + stocks in EUR"], downcast="float", errors="coerce")
df["Bonus"].fillna(0, inplace=True)
df["target"] = df["Yearly brutto salary (without bonus and stocks) in EUR"] + df["Bonus"]
df.drop(columns={"Timestamp","Yearly brutto salary (without bonus and stocks) in EUR", "Yearly bonus + stocks in EUR", "Bonus"}, inplace=True)
df['Total years of experience'] = pd.to_numeric(df['Total years of experience'], downcast="float", errors='coerce')
df['Years of experience in Germany'] = pd.to_numeric(df['Years of experience in Germany'], downcast="float", errors='coerce')
df['Number of vacation days'] = pd.to_numeric(df['Number of vacation days'], downcast="float", errors='coerce')
df['Gender'] = df['Gender'].astype('string')
df['City'] = df['City'].astype('string')
df['Seniority level'] = df['Seniority level'].astype('string')
df['Main language at work'] = df['Main language at work'].astype('string')
df['Employment status'] = df['Employment status'].astype('string')
df['Company size'] = df['Company size'].astype('string')
df['Сontract duration'] = df['Сontract duration'].astype('string')
df['Your main technology / programming language'] = df['Your main technology / programming language'].astype('string')
df['Other technologies/programming languages you use often'] = df['Other technologies/programming languages you use often'].astype('string')
df['Company Type'] = df['Other technologies/programming languages you use often'].astype('Company Type')



TypeError: data type 'Company Type' not understood

In [ ]:
df["Your main technology / programming language"].value_counts()

In [ ]:
df["Other technologies/programming languages you use often"].value_counts()

In [ ]:
df["Number of vacation days"].value_counts()

In [ ]:
df["Years of experience in Germany"].value_counts()

In [ ]:
df["Сontract duration"].value_counts()

In [ ]:
df["Age"].value_counts()


In [ ]:
df["Company type"].value_counts()

In [ ]:
df["Company size"].value_counts()

In [ ]:
df["Employment status"].value_counts()

In [ ]:
df["Total years of experience"].value_counts()

In [ ]:
df["Main language at work"].value_counts()

In [ ]:
df["Seniority level"].value_counts()

In [ ]:
df["Gender"].value_counts()

In [ ]:
def replace_low_freq(d, col, threshold=10, replacement='other'):
    value_counts = d[col].value_counts() # Specific column 
    to_remove = value_counts[value_counts <= threshold].index
    tmp = d[col].replace(to_replace=to_remove, value=replacement)
    return tmp

In [ ]:
# Replace low frequency values with one new column
df["Your main technology / programming language"] = replace_low_freq(df, "Your main technology / programming language", 10, "Other")
df["Other technologies/programming languages you use often"] = replace_low_freq(df, "Other technologies/programming languages you use often", 10, "Other")
df["Seniority level"] = replace_low_freq(df, "Seniority level", 10, "Other")
df["Main language at work"] = replace_low_freq(df, "Main language at work", 10, "Other")
df["Employment status"] = replace_low_freq(df, "Employment status", 7, "Other")
df["Company type"] = replace_low_freq(df, "Company type", 10, "Other")


In [ ]:
# Replace NaN and other values
df["Your main technology / programming language"].replace({"Javascript": "JavaScript", "Python ":"Python", "python": "Python", np.nan : "None"}, inplace=True)
df["Other technologies/programming languages you use often"].replace({np.nan : "None"}, inplace=True)
df["Number of vacation days"].replace({np.nan : "0", "Unlimited": np.inf, "unlimited": np.inf}, inplace=True)
df["Your main technology / programming language"].value_counts()
df["Years of experience in Germany"].replace({np.nan : 0, "," : "." }, inplace=True,  regex=True)
df["Gender"].replace({np.nan : "Diverse"}, inplace=True)
df["Main language at work"].replace({np.nan : "Other"}, inplace=True )
df["Total years of experience"].replace(np.nan , 0, inplace=True)
df["Seniority level"].replace({np.nan : "Other"}, inplace=True)
df["Employment status"].replace({np.nan : "Other"}, inplace=True)
df["Company type"].replace({np.nan : "Other"}, inplace=True)
df["Company size"].replace({np.nan : "up to 10"}, inplace=True)
df["Сontract duration"].replace({np.nan : "None"}, inplace=True)
df["Age"].replace({np.nan : df['Age'].mean().round(0)}, inplace=True)


In [ ]:
# Drop columns
df = df.drop(columns = ["Position"],inplace=True)

In [ ]:
# Double check for nulls
df.isnull().sum().sort_values()

In [ ]:
df.info()

In [ ]:
# See relationship and see outliers
sns.pairplot(df[['Age', 'Years of experience in Germany', 'Total years of experience']])

In [ ]:
#Remove outliers
#df = df[df['Age'] <= 65.0]

In [ ]:
"""
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier 

model = RandomForestClassifier(n_jobs=-1)
pipe_model = LogisticRegression(max_iter=10000, solver="saga")

min_features_to_select = 1  # Minimum number of features to consider
rfecv_pipe = RFECV(
    estimator=pipe_model,
    step=1,
    cv=3,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
)

X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y)


pipeline_steps = [("fs", rfecv_pipe),
                ('model', model) ]
pipe = Pipeline(pipeline_steps)
pipe.fit(X_train_, y_train_)
pipe.score(X_test_,y_test_)
"""

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

numeric_features = ["Age", "Total years of experience", "Years of experience in Germany"]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["Gender", "City", "Seniority level", "Your main technology / programming language", "Other technologies/programming languages you use often",
"Number of vacation days", "Employment status", "Сontract duration", "Main language at work", "Company size", "Company type"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

y = df["target"]
X = df.drop(columns={"target"})
print(X.shape, y.shape)

clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))
